# 랭체인(LangChain) SQL Query validation 예제
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## Reference : https://python.langchain.com/docs/use_cases/sql/query_checking

![](https://python.langchain.com/assets/images/sql_usecase-d432701261f05ab69b38576093718cf3.png)

# Sample SQL DB 다운로드

## Reference : https://www.sqlitetutorial.net/sqlite-sample-database/

![](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

In [ ]:
!wget https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -O chinook.zip

--2024-03-11 14:17:14--  https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 104.21.30.141, 172.67.172.250, 2606:4700:3037::6815:1e8d, ...
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|104.21.30.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘chinook.zip’

chinook.zip         100%[===================>] 298.43K  --.-KB/s    in 0.02s   

2024-03-11 14:17:14 (13.1 MB/s) - ‘chinook.zip’ saved [305596/305596]



In [ ]:
!unzip chinook.zip

Archive:  chinook.zip
  inflating: chinook.db              


# LangChain 라이브러리 설치

In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchain-openai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

# chinook.db 불러오기

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM artists LIMIT 10;")

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# Query checker

* 가장 간단한 전략은 모델에게 원본 쿼리에서 흔한 실수를 확인하도록 요청하는 것입니다. 다음과 같은 SQL 쿼리 체인을 가정해 보겠습니다.







In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=OPENAI_KEY)
chain = create_sql_query_chain(llm, db)

* 그리고 우리는 그 출력을 검증하고 싶습니다. 체인을 두 번째 프롬프트와 모델 호출로 확장함으로써 그렇게 할 수 있습니다:

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """사용자의 {dialect} 쿼리를 다음과 같은 흔한 실수에 대해 다시 확인하세요:
- NULL 값에 NOT IN을 사용하는 경우
- UNION을 사용할 때 UNION ALL을 사용해야 하는 경우
- 배타적 범위에 BETWEEN을 사용하는 경우
- 조건문에서 데이터 타입 불일치
- 식별자를 올바르게 인용하는 경우
- 함수에 올바른 인수 수를 사용하는 경우
- 올바른 데이터 타입으로 캐스팅하는 경우
- 조인에 적합한 컬럼을 사용하는 경우

위의 실수 중 어느 것이라도 있다면, 쿼리를 다시 작성하세요. 실수가 없다면, 원본 쿼리를 그대로 재생산하세요.

최종 SQL 쿼리만 출력하세요."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [ ]:
query = full_chain.invoke(
    {
        "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
    }
)
query

In [ ]:
db.run(query)

'[(6.390909090909091,)]'

* 이 접근법의 명백한 단점은 쿼리를 생성하기 위해 하나 대신 두 번의 모델 호출을 해야 한다는 것입니다. 이를 해결하기 위해 우리는 쿼리 생성과 쿼리 검사를 단일 모델 호출에서 수행하려고 시도할 수 있습니다:

In [ ]:
system = """당신은 {dialect} 전문가입니다. 주어진 입력 질문에 대해 문법적으로 정확한 {dialect} 쿼리를 작성하십시오.
사용자가 질문에서 특정한 예시의 수를 지정하지 않은 경우, {dialect}에 따라 LIMIT 절을 사용하여 최대 {top_k}개의 결과를 조회하십시오. 데이터베이스에서 가장 유익한 데이터를 반환하기 위해 결과를 정렬할 수 있습니다.
테이블의 모든 컬럼을 조회해서는 안 됩니다. 질문에 답하기 위해 필요한 컬럼만 조회해야 합니다. 각 컬럼 이름을 이중 인용부호(")로 감싸 구분된 식별자로 표시하십시오.
아래 테이블에서 볼 수 있는 컬럼 이름만 사용해야 합니다. 존재하지 않는 컬럼을 조회하지 않도록 주의하십시오. 또한, 어떤 컬럼이 어느 테이블에 있는지 주의 깊게 살펴보십시오.
질문에 "오늘"이 포함된 경우 현재 날짜를 얻기 위해 date('now') 함수를 사용하십시오.

다음 테이블만 사용하세요:
{table_info}"

쿼리의 초안을 작성하세요. 그런 다음, {dialect} 쿼리에서 다음을 포함한 흔한 실수를 다시 확인하세요:
- NULL 값에 NOT IN을 사용하는 경우
- UNION을 사용할 때 UNION ALL을 사용해야 하는 경우
- 배타적 범위에 BETWEEN을 사용하는 경우
- 조건문에서 데이터 타입 불일치
- 식별자를 올바르게 인용하는 경우
- 함수에 올바른 인수 수를 사용하는 경우
- 올바른 데이터 타입으로 캐스팅하는 경우
- 조인에 적합한 컬럼을 사용하는 경우

아래 형식을 사용하세요:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>
"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}")]
).partial(dialect=db.dialect)


def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]

chain = create_sql_query_chain(llm, db, prompt=prompt) | parse_final_answer
prompt.pretty_print()

================================ System Message ================================

당신은 {dialect} 전문가입니다. 주어진 입력 질문에 대해 문법적으로 정확한 {dialect} 쿼리를 작성하십시오.
사용자가 질문에서 특정한 예시의 수를 지정하지 않은 경우, {dialect}에 따라 LIMIT 절을 사용하여 최대 {top_k}개의 결과를 조회하십시오. 데이터베이스에서 가장 유익한 데이터를 반환하기 위해 결과를 정렬할 수 있습니다.
테이블의 모든 컬럼을 조회해서는 안 됩니다. 질문에 답하기 위해 필요한 컬럼만 조회해야 합니다. 각 컬럼 이름을 이중 인용부호(")로 감싸 구분된 식별자로 표시하십시오.
아래 테이블에서 볼 수 있는 컬럼 이름만 사용해야 합니다. 존재하지 않는 컬럼을 조회하지 않도록 주의하십시오. 또한, 어떤 컬럼이 어느 테이블에 있는지 주의 깊게 살펴보십시오.
질문에 "오늘"이 포함된 경우 현재 날짜를 얻기 위해 date('now') 함수를 사용하십시오.

다음 테이블만 사용하세요:
{table_info}"

쿼리의 초안을 작성하세요. 그런 다음, {dialect} 쿼리에서 다음을 포함한 흔한 실수를 다시 확인하세요:
- NULL 값에 NOT IN을 사용하는 경우
- UNION을 사용할 때 UNION ALL을 사용해야 하는 경우
- 배타적 범위에 BETWEEN을 사용하는 경우
- 조건문에서 데이터 타입 불일치
- 식별자를 올바르게 인용하는 경우
- 함수에 올바른 인수 수를 사용하는 경우
- 올바른 데이터 타입으로 캐스팅하는 경우
- 조인에 적합한 컬럼을 사용하는 경우

아래 형식을 사용하세요:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>


================================ Human Message =================================

{input}

In [ ]:
query = chain.invoke(
    {
        "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
    }
)
query

In [ ]:
db.run(query)

'[(6.632999999999998,)]'